In [1]:
# import tensorflow as tf
# print(tf.__version__)
# !pip install tensorflow==2.15.1
# print(tf.__version__)

In [2]:
# pip install pyswarms scikit-learn

2

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset directory
train_data_dir = '/kaggle/input/unbala/unbalanced_split/train'
test_data_dir = '/kaggle/input/unbala/unbalanced_split/test'

# Set the hyperparameters
# img_width = 224
# img_height = 224
img_width = 224
img_height = 224
batch_size = 32
seed = 123

# Define the data generators with preprocessing steps
train_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.3)
test_data_generator = ImageDataGenerator(rescale=1./255)

# Load the training and validation datasets using the data generator
train_generator = train_data_generator.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    color_mode='rgb',
    class_mode='categorical',  # Set class_mode to 'categorical' for 3 classes
    subset='training'
)

validation_generator = train_data_generator.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
    color_mode='rgb',
    class_mode='categorical',  # Set class_mode to 'categorical' for 3 classes
    subset='validation'
)

# Load the test dataset using the data generator
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False,
    color_mode='rgb',
    class_mode='categorical'  # Set class_mode to 'categorical' for 3 classes
)


2024-08-31 10:28:26.156118: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-31 10:28:26.156198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-31 10:28:26.157894: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 2352 images belonging to 3 classes.
Found 1007 images belonging to 3 classes.
Found 1442 images belonging to 3 classes.


In [4]:
import numpy as np

# Calculate the total number of samples in each set
train_samples = train_generator.samples
validation_samples = validation_generator.samples
test_samples = test_generator.samples

# Preallocate arrays to hold the data
X_train_features = np.zeros((train_samples, img_width, img_height, 3), dtype=np.float32)
y_train = np.zeros((train_samples, 3), dtype=np.float32)  # Assuming 3 classes for one-hot encoding

X_val_features = np.zeros((validation_samples, img_width, img_height, 3), dtype=np.float32)
y_val = np.zeros((validation_samples, 3), dtype=np.float32)  # Assuming 3 classes for one-hot encoding

X_test_features = np.zeros((test_samples, img_width, img_height, 3), dtype=np.float32)
y_test = np.zeros((test_samples, 3), dtype=np.float32)  # Assuming 3 classes for one-hot encoding

# Function to convert generator data to numpy arrays
def generator_to_numpy(generator, X_array, y_array):
    i = 0
    for X_batch, y_batch in generator:
        batch_size = X_batch.shape[0]
        X_array[i:i+batch_size] = X_batch
        y_array[i:i+batch_size] = y_batch
        i += batch_size
        if i >= X_array.shape[0]:
            break
    return X_array, y_array

# Convert training and validation data
X_train_features, y_train = generator_to_numpy(train_generator, X_train_features, y_train)
X_val_features, y_val = generator_to_numpy(validation_generator, X_val_features, y_val)

# Convert test data
X_test_features, y_test = generator_to_numpy(test_generator, X_test_features, y_test)

print(f"X_train_features shape: {X_train_features.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val_features shape: {X_val_features.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test_features shape: {X_test_features.shape}")
print(f"y_test shape: {y_test.shape}")


X_train_features shape: (2352, 224, 224, 3)
y_train shape: (2352, 3)
X_val_features shape: (1007, 224, 224, 3)
y_val shape: (1007, 3)
X_test_features shape: (1442, 224, 224, 3)
y_test shape: (1442, 3)


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import load_model

# Load the trained DenseNet121 model
model = load_model('/kaggle/input/best/tensorflow2/default/1/densenet_best.h5')

# Get the DenseNet121 sub-model
densenet_model = model.get_layer('densenet121')

# Get the output of the last convolutional block in DenseNet121
# Replace 'conv5_block16_concat' with the correct layer name if necessary
x = densenet_model.get_layer('conv5_block16_concat').output

# Add GlobalAveragePooling2D to reduce spatial dimensions
x = GlobalAveragePooling2D()(x)

# Create the feature extraction model using the input of the DenseNet121 sub-model
feature_extractor = Model(inputs=densenet_model.input, outputs=x)

# Extract features from your data
X_train_features = feature_extractor.predict(train_generator)
X_val_features = feature_extractor.predict(validation_generator)
X_test_features = feature_extractor.predict(test_generator)

# Print the shape of the extracted features
print(f"Train features shape: {X_train_features.shape}")
print(f"Validation features shape: {X_val_features.shape}")
print(f"Test features shape: {X_test_features.shape}")

46/46 [==============================] - 6s 121ms/step
Train features shape: (2352, 1024)
Validation features shape: (1007, 1024)
Test features shape: (1442, 1024)


In [6]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
X_train_scaled = scaler.fit_transform(X_train_features)
X_val_scaled = scaler.transform(X_val_features)
X_test_scaled = scaler.transform(X_test_features)

In [7]:
X_train_scaled.shape

(2352, 1024)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the FFNN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')  # Assuming you have 3 classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train_scaled, np.argmax(y_train, axis=1),
                    validation_data=(X_val_scaled, np.argmax(y_val, axis=1)),
                    epochs=50, batch_size=32, callbacks=[early_stopping])

# Evaluate on the validation set
val_loss, val_accuracy = model.evaluate(X_val_scaled, np.argmax(y_val, axis=1))
print(f"Validation Accuracy: {val_accuracy}")

# Evaluate on the test set (if available)
test_loss, test_accuracy = model.evaluate(X_test_scaled, np.argmax(y_test, axis=1))
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50


I0000 00:00:1725100160.503796    1360 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - 3s 8ms/step - loss: 0.3850 - accuracy: 0.8397 - val_loss: 0.0078 - val_accuracy: 0.9980
Epoch 2/50
74/74 [==============================] - 0s 5ms/step - loss: 0.0848 - accuracy: 0.9702 - val_loss: 0.0022 - val_accuracy: 0.9990
Epoch 3/50
74/74 [==============================] - 0s 5ms/step - loss: 0.0472 - accuracy: 0.9843 - val_loss: 0.0013 - val_accuracy: 0.9990
Epoch 4/50
74/74 [==============================] - 0s 5ms/step - loss: 0.0393 - accuracy: 0.9855 - val_loss: 0.0047 - val_accuracy: 0.9990
Epoch 5/50
74/74 [==============================] - 0s 5ms/step - loss: 0.0371 - accuracy: 0.9898 - val_loss: 0.0046 - val_accuracy: 0.9990
Epoch 6/50
74/74 [==============================] - 0s 5ms/step - loss: 0.0201 - accuracy: 0.9923 - val_loss: 0.0041 - val_accuracy: 0.9990
Epoch 7/50
74/74 [==============================] - 0s 5ms/step - loss: 0.0186 - accuracy: 0.9919 - val_loss: 0.0030 - val_accuracy: 0.9990
Epoch 8/50
74/74 [=============

In [15]:
X_train_scaled.shape

(2352, 1024)

In [16]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the SVC model
svc_model = SVC(kernel='linear', random_state=42)

# Train the SVC model on non-reduced extracted features
svc_model.fit(X_train_scaled, np.argmax(y_train, axis=1))

# Evaluate the model on the validation set
y_val_pred = svc_model.predict(X_val_scaled)
val_accuracy = accuracy_score(np.argmax(y_val, axis=1), y_val_pred)
print(f"Validation Accuracy with SVC: {val_accuracy}")

# Evaluate the model on the test set
y_test_pred = svc_model.predict(X_test_scaled)
test_accuracy = accuracy_score(np.argmax(y_test, axis=1), y_test_pred)
print(f"Test Accuracy with SVC: {test_accuracy}")


Validation Accuracy with SVC: 0.9990069513406157
Test Accuracy with SVC: 0.9479889042995839


In [9]:
import numpy as np
import pyswarms as ps
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score

# Objective function for PSO-GA hybrid
def objective_function(particles, X_train_scaled, y_train, X_val_scaled, y_val):
    n_particles = particles.shape[0]
    scores = []
    
    for i in range(n_particles):
        # Convert the particle to a boolean mask
        mask = particles[i] > 0.5
        
        if mask.sum() == 0:
            # Penalize if no feature is selected
            scores.append(np.inf)
            continue
        
        X_train_selected = X_train_scaled[:, mask]
        X_val_selected = X_val_scaled[:, mask]
        
        print(f"Selected features: {mask.sum()}")
        print(f"X_train_selected shape: {X_train_selected.shape}")
        print(f"X_val_selected shape: {X_val_selected.shape}")
        
        # Train an SVC classifier
        clf = SVC(kernel='linear', random_state=42)
        clf.fit(X_train_selected, y_train)
        
        # Predict on the validation set
        y_pred = clf.predict(X_val_selected)
        
        # Calculate the balanced accuracy
        score = balanced_accuracy_score(y_val, y_pred)
        
        # Penalize the score based on the number of features selected
        penalty = mask.sum() / X_train_scaled.shape[1]  # Fraction of features selected
        scores.append(penalty - score)
    
    return np.array(scores)

# Initialize the parameters for PSO
options = {'c1': 0.7, 'c2': 0.5, 'w': 0.9}
dimensions = X_train_scaled.shape[1]  # Number of features
n_particles = 30  # Number of particles

# Initialize the PSO optimizer
pso_optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=dimensions, options=options)

# Run initial PSO optimization
best_cost, best_solution = pso_optimizer.optimize(
    objective_function, 
    iters=20, 
    X_train_scaled=X_train_scaled, y_train=np.argmax(y_train, axis=1), 
    X_val_scaled=X_val_scaled, y_val=np.argmax(y_val, axis=1)
)

# Convert the best solution from PSO into a GA population
ga_population = np.array([best_solution for _ in range(n_particles)])

# GA Parameters
mutation_rate = 0.1
crossover_rate = 0.7
num_generations = 50

# Run GA optimization
for generation in range(num_generations):
    new_population = []

    # Crossover and Mutation
    for i in range(0, n_particles, 2):
        parent1 = ga_population[i]
        parent2 = ga_population[(i+1) % n_particles]

        if np.random.rand() < crossover_rate:
            # One-point crossover
            crossover_point = np.random.randint(1, dimensions)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
        else:
            child1, child2 = parent1.copy(), parent2.copy()

        # Mutation
        if np.random.rand() < mutation_rate:
            mutation_index = np.random.randint(0, dimensions)
            child1[mutation_index] = 1 - child1[mutation_index]  # Flip bit
        if np.random.rand() < mutation_rate:
            mutation_index = np.random.randint(0, dimensions)
            child2[mutation_index] = 1 - child2[mutation_index]  # Flip bit

        new_population.extend([child1, child2])

    ga_population = np.array(new_population)

    # Evaluate the population
    fitness = objective_function(ga_population, X_train_scaled, np.argmax(y_train, axis=1), X_val_scaled, np.argmax(y_val, axis=1))
    
    # Select the best individual
    best_index = np.argmin(fitness)
    best_cost_ga = fitness[best_index]
    best_solution_ga = ga_population[best_index]
    
    print(f"Generation {generation + 1}/{num_generations}, Best Cost: {best_cost_ga}")

# Final best solution from the GA
best_mask = best_solution_ga > 0.5
X_train_selected = X_train_scaled[:, best_mask]
X_val_selected = X_val_scaled[:, best_mask]

print(f"Best selected features: {best_mask.sum()}")
print(f"Final Best Cost: {best_cost_ga}")


2024-08-31 10:29:30,114 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.7, 'c2': 0.5, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/20

Selected features: 498
X_train_selected shape: (2352, 498)
X_val_selected shape: (1007, 498)
Selected features: 479
X_train_selected shape: (2352, 479)
X_val_selected shape: (1007, 479)
Selected features: 493
X_train_selected shape: (2352, 493)
X_val_selected shape: (1007, 493)
Selected features: 496
X_train_selected shape: (2352, 496)
X_val_selected shape: (1007, 496)
Selected features: 467
X_train_selected shape: (2352, 467)
X_val_selected shape: (1007, 467)
Selected features: 506
X_train_selected shape: (2352, 506)
X_val_selected shape: (1007, 506)
Selected features: 521
X_train_selected shape: (2352, 521)
X_val_selected shape: (1007, 521)
Selected features: 548
X_train_selected shape: (2352, 548)
X_val_selected shape: (1007, 548)
Selected features: 516
X_train_selected shape: (2352, 516)
X_val_selected shape: (1007, 516)
Selected features: 486
X_train_selected shape: (2352, 486)
X_val_selected shape: (1007, 486)
Selected features: 490
X_train_selected shape: (2352, 490)
X_val_selec

pyswarms.single.global_best:   5%|▌         |1/20, best_cost=-0.543

Selected features: 917
X_train_selected shape: (2352, 917)
X_val_selected shape: (1007, 917)
Selected features: 895
X_train_selected shape: (2352, 895)
X_val_selected shape: (1007, 895)
Selected features: 908
X_train_selected shape: (2352, 908)
X_val_selected shape: (1007, 908)
Selected features: 905
X_train_selected shape: (2352, 905)
X_val_selected shape: (1007, 905)
Selected features: 868
X_train_selected shape: (2352, 868)
X_val_selected shape: (1007, 868)
Selected features: 921
X_train_selected shape: (2352, 921)
X_val_selected shape: (1007, 921)
Selected features: 917
X_train_selected shape: (2352, 917)
X_val_selected shape: (1007, 917)
Selected features: 912
X_train_selected shape: (2352, 912)
X_val_selected shape: (1007, 912)
Selected features: 913
X_train_selected shape: (2352, 913)
X_val_selected shape: (1007, 913)
Selected features: 894
X_train_selected shape: (2352, 894)
X_val_selected shape: (1007, 894)
Selected features: 912
X_train_selected shape: (2352, 912)
X_val_selec

pyswarms.single.global_best:  10%|█         |2/20, best_cost=-0.543

...

Selected features: 17
X_train_selected shape: (2352, 17)
X_val_selected shape: (1007, 17)
Selected features: 39
X_train_selected shape: (2352, 39)
X_val_selected shape: (1007, 39)
Selected features: 24
X_train_selected shape: (2352, 24)
X_val_selected shape: (1007, 24)
Selected features: 30
X_train_selected shape: (2352, 30)
X_val_selected shape: (1007, 30)
Selected features: 24
X_train_selected shape: (2352, 24)
X_val_selected shape: (1007, 24)
Selected features: 17
X_train_selected shape: (2352, 17)
X_val_selected shape: (1007, 17)
Selected features: 17
X_train_selected shape: (2352, 17)
X_val_selected shape: (1007, 17)
Selected features: 18
X_train_selected shape: (2352, 18)
X_val_selected shape: (1007, 18)
Selected features: 24
X_train_selected shape: (2352, 24)
X_val_selected shape: (1007, 24)
Selected features: 18
X_train_selected shape: (2352, 18)
X_val_selected shape: (1007, 18)
Selected features: 13
X_train_selected shape: (2352, 13)
X_val_selected shape: (1007, 13)
Selected f

pyswarms.single.global_best:  95%|█████████▌|19/20, best_cost=-0.983

Selected features: 28
X_train_selected shape: (2352, 28)
X_val_selected shape: (1007, 28)
Selected features: 22
X_train_selected shape: (2352, 22)
X_val_selected shape: (1007, 22)
Selected features: 21
X_train_selected shape: (2352, 21)
X_val_selected shape: (1007, 21)
Selected features: 19
X_train_selected shape: (2352, 19)
X_val_selected shape: (1007, 19)
Selected features: 22
X_train_selected shape: (2352, 22)
X_val_selected shape: (1007, 22)
Selected features: 36
X_train_selected shape: (2352, 36)
X_val_selected shape: (1007, 36)
Selected features: 21
X_train_selected shape: (2352, 21)
X_val_selected shape: (1007, 21)
Selected features: 25
X_train_selected shape: (2352, 25)
X_val_selected shape: (1007, 25)
Selected features: 20
X_train_selected shape: (2352, 20)
X_val_selected shape: (1007, 20)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected features: 19
X_train_selected shape: (2352, 19)
X_val_selected shape: (1007, 19)
Selected f

pyswarms.single.global_best: 100%|██████████|20/20, best_cost=-0.983
2024-08-31 10:30:08,205 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.9825385519048883, best pos: [-1.4427339  -1.16429274 -1.61300198 ...  0.78964742 -1.23307558
 -1.66949696]


Selected features: 16
X_train_selected shape: (2352, 16)
X_val_selected shape: (1007, 16)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected features: 16
X_train_selected shape: (2352, 16)
X_val_selected shape: (1007, 16)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected features: 16
X_train_selected shape: (2352, 16)
X_val_selected shape: (1007, 16)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected features: 15
X_train_selected shape: (2352, 15)
X_val_selected shape: (1007, 15)
Selected f

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Apply the best_mask to select the best features from the training and validation data
X_train_best_features = X_train_scaled[:, best_mask]
X_val_best_features = X_val_scaled[:, best_mask]

# Print the shapes to verify
print(f"X_train_best_features shape: {X_train_best_features.shape}")
print(f"X_val_best_features shape: {X_val_best_features.shape}")

# Split the training data into train and test for evaluating the FFNN
X_train_ffnn, X_test_ffnn, y_train_ffnn, y_test_ffnn = train_test_split(X_train_best_features, y_train, test_size=0.2, random_state=42)

# Define the FFNN model with Dropout layers
model = Sequential()
model.add(Dense(64, input_shape=(X_train_best_features.shape[1],), activation='relu'))
model.add(Dropout(0.3))  # Add Dropout with a rate of 0.5
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))  # Add another Dropout layer
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, min_delta=0.001)

# Train the model
history = model.fit(X_train_ffnn, np.argmax(y_train_ffnn, axis=1), 
                    validation_data=(X_val_best_features, np.argmax(y_val, axis=1)), 
                    epochs=50, batch_size=32, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_ffnn, np.argmax(y_test_ffnn, axis=1))

print(f"Test Accuracy: {test_accuracy}")


X_train_best_features shape: (2352, 16)
X_val_best_features shape: (1007, 16)
Epoch 1/50
59/59 [==============================] - 1s 7ms/step - loss: 0.7195 - accuracy: 0.7198 - val_loss: 0.3260 - val_accuracy: 0.9623
Epoch 2/50
59/59 [==============================] - 0s 5ms/step - loss: 0.2615 - accuracy: 0.9436 - val_loss: 0.0829 - val_accuracy: 0.9911
Epoch 3/50
59/59 [==============================] - 0s 4ms/step - loss: 0.1171 - accuracy: 0.9734 - val_loss: 0.0380 - val_accuracy: 0.9940
Epoch 4/50
59/59 [==============================] - 0s 4ms/step - loss: 0.0805 - accuracy: 0.9787 - val_loss: 0.0243 - val_accuracy: 0.9960
Epoch 5/50
59/59 [==============================] - 0s 4ms/step - loss: 0.0609 - accuracy: 0.9793 - val_loss: 0.0187 - val_accuracy: 0.9960
Epoch 6/50
59/59 [==============================] - 0s 4ms/step - loss: 0.0481 - accuracy: 0.9872 - val_loss: 0.0156 - val_accuracy: 0.9950
Epoch 7/50
59/59 [==============================] - 0s 4ms/step - loss: 0.0353 - a

In [14]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Apply the best_mask to select the best features from the training and validation data
X_train_best_features = X_train_scaled[:, best_mask]
X_val_best_features = X_val_scaled[:, best_mask]

# Print the shapes to verify
print(f"X_train_best_features shape: {X_train_best_features.shape}")
print(f"X_val_best_features shape: {X_val_best_features.shape}")

# Split the training data into train and test sets
X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(X_train_best_features, np.argmax(y_train, axis=1), test_size=0.2, random_state=42)

# Initialize the SVC model
svc_model = SVC(kernel='linear', random_state=42)

# Train the SVC model
svc_model.fit(X_train_svc, y_train_svc)

# Evaluate the model on the validation set
y_val_pred = svc_model.predict(X_val_best_features)
val_accuracy = accuracy_score(np.argmax(y_val, axis=1), y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")

# Evaluate the model on the test set
y_test_pred = svc_model.predict(X_test_svc)
test_accuracy = accuracy_score(y_test_svc, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")


X_train_best_features shape: (2352, 16)
X_val_best_features shape: (1007, 16)
Validation Accuracy: 0.9950347567030785
Test Accuracy: 0.9915074309978769
